Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
batch_size = 128
lambda_value = 0.01
graph = tf.Graph()
h1_size = 1024

with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, h1_size]))
  biases = tf.Variable(tf.zeros([h1_size]))
  h1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)

  # Training computation.
  weight_2 = tf.Variable(
    tf.truncated_normal([h1_size, num_labels]))
  biases_2 =  tf.Variable(tf.zeros([num_labels]))

  logits = tf.matmul(h1, weight_2) + biases_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)+
    lambda_value*tf.nn.l2_loss(weights) +
    lambda_value*tf.nn.l2_loss(biases) +
    lambda_value*tf.nn.l2_loss(weight_2) +
    lambda_value*tf.nn.l2_loss(biases_2))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases), weight_2) + biases_2)
  test_prediction = tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases), weight_2) + biases_2)

In [6]:
num_steps = 5001


with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3465.482666
Minibatch accuracy: 11.7%
Validation accuracy: 25.9%
Minibatch loss at step 500: 21.247417
Minibatch accuracy: 85.2%
Validation accuracy: 84.2%
Minibatch loss at step 1000: 0.828088
Minibatch accuracy: 85.9%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 0.605818
Minibatch accuracy: 88.3%
Validation accuracy: 82.3%
Minibatch loss at step 2000: 0.595056
Minibatch accuracy: 85.2%
Validation accuracy: 82.7%
Minibatch loss at step 2500: 0.746862
Minibatch accuracy: 82.8%
Validation accuracy: 82.1%
Minibatch loss at step 3000: 0.813478
Minibatch accuracy: 79.7%
Validation accuracy: 82.8%
Minibatch loss at step 3500: 0.771447
Minibatch accuracy: 83.6%
Validation accuracy: 83.4%
Minibatch loss at step 4000: 0.818430
Minibatch accuracy: 82.0%
Validation accuracy: 83.0%
Minibatch loss at step 4500: 0.829669
Minibatch accuracy: 80.5%
Validation accuracy: 83.8%
Minibatch loss at step 5000: 0.698709
Minibatch accuracy: 85.2%
Validation acc

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [7]:
batch_size = 128
lambda_value = 0.01
graph = tf.Graph()
h1_size = 1024

with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, h1_size]))
  biases = tf.Variable(tf.zeros([h1_size]))
  h1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)

  # Training computation.
  weight_2 = tf.Variable(
    tf.truncated_normal([h1_size, num_labels]))
  biases_2 =  tf.Variable(tf.zeros([num_labels]))

  logits = tf.matmul(h1, weight_2) + biases_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)+
    lambda_value*tf.nn.l2_loss(weights) +
    lambda_value*tf.nn.l2_loss(biases) +
    lambda_value*tf.nn.l2_loss(weight_2) +
    lambda_value*tf.nn.l2_loss(biases_2))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases), weight_2) + biases_2)
  test_prediction = tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases), weight_2) + biases_2)

In [8]:
num_steps = 5001


with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3511.535156
Minibatch accuracy: 1.6%
Validation accuracy: 34.9%
Minibatch loss at step 500: 21.212648
Minibatch accuracy: 83.6%
Validation accuracy: 84.0%
Minibatch loss at step 1000: 0.822376
Minibatch accuracy: 85.2%
Validation accuracy: 81.7%
Minibatch loss at step 1500: 0.602611
Minibatch accuracy: 88.3%
Validation accuracy: 82.4%
Minibatch loss at step 2000: 0.597667
Minibatch accuracy: 85.2%
Validation accuracy: 82.6%
Minibatch loss at step 2500: 0.745797
Minibatch accuracy: 82.8%
Validation accuracy: 82.1%
Minibatch loss at step 3000: 0.812814
Minibatch accuracy: 79.7%
Validation accuracy: 83.0%
Minibatch loss at step 3500: 0.771994
Minibatch accuracy: 83.6%
Validation accuracy: 83.5%
Minibatch loss at step 4000: 0.812915
Minibatch accuracy: 82.8%
Validation accuracy: 83.0%
Minibatch loss at step 4500: 0.831939
Minibatch accuracy: 80.5%
Validation accuracy: 83.8%
Minibatch loss at step 5000: 0.697125
Minibatch accuracy: 85.2%
Validation accu

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [9]:
batch_size = 128
lambda_value = 0.01
graph = tf.Graph()
h1_size = 1024

with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, h1_size]))
  biases = tf.Variable(tf.zeros([h1_size]))
  h1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)

  keep_prob = tf.placeholder(tf.float32)
  h1_dropout = tf.nn.dropout(h1, keep_prob)
   

  # Training computation.
  weight_2 = tf.Variable(
    tf.truncated_normal([h1_size, num_labels]))
  biases_2 =  tf.Variable(tf.zeros([num_labels]))

  logits = tf.matmul(h1_dropout, weight_2) + biases_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)+
    lambda_value*tf.nn.l2_loss(weights) +
    lambda_value*tf.nn.l2_loss(biases) +
    lambda_value*tf.nn.l2_loss(weight_2) +
    lambda_value*tf.nn.l2_loss(biases_2))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases), weight_2) + biases_2)
  test_prediction = tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases), weight_2) + biases_2)

In [10]:
num_steps = 5001

keep_prob_value = 1
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {
        tf_train_dataset : batch_data, tf_train_labels : batch_labels,
        keep_prob : keep_prob_value}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3507.067139
Minibatch accuracy: 14.8%
Validation accuracy: 36.5%
Minibatch loss at step 500: 21.270918
Minibatch accuracy: 83.6%
Validation accuracy: 83.7%
Minibatch loss at step 1000: 0.815830
Minibatch accuracy: 85.2%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 0.603755
Minibatch accuracy: 87.5%
Validation accuracy: 82.4%
Minibatch loss at step 2000: 0.594837
Minibatch accuracy: 84.4%
Validation accuracy: 82.7%
Minibatch loss at step 2500: 0.746204
Minibatch accuracy: 84.4%
Validation accuracy: 82.1%
Minibatch loss at step 3000: 0.810672
Minibatch accuracy: 80.5%
Validation accuracy: 82.8%
Minibatch loss at step 3500: 0.769078
Minibatch accuracy: 83.6%
Validation accuracy: 83.3%
Minibatch loss at step 4000: 0.816118
Minibatch accuracy: 82.8%
Validation accuracy: 83.0%
Minibatch loss at step 4500: 0.831506
Minibatch accuracy: 80.5%
Validation accuracy: 83.8%
Minibatch loss at step 5000: 0.698571
Minibatch accuracy: 85.2%
Validation acc

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [11]:
def batch_norm_wrapper(inputs, is_training, decay = 0.999):

    # Small epsilon value for the BN transform
    epsilon = 1e-3

    scale = tf.Variable(tf.ones([inputs.get_shape()[-1]]))
    beta = tf.Variable(tf.zeros([inputs.get_shape()[-1]]))
    pop_mean = tf.Variable(tf.zeros([inputs.get_shape()[-1]]), trainable=False)
    pop_var = tf.Variable(tf.ones([inputs.get_shape()[-1]]), trainable=False)

    if is_training:
        batch_mean, batch_var = tf.nn.moments(inputs,[0])
        train_mean = tf.assign(pop_mean,
                               pop_mean * decay + batch_mean * (1 - decay))
        train_var = tf.assign(pop_var,
                              pop_var * decay + batch_var * (1 - decay))
        with tf.control_dependencies([train_mean, train_var]):
            return tf.nn.batch_normalization(inputs,
                batch_mean, batch_var, beta, scale, epsilon)
    else:
        return tf.nn.batch_normalization(inputs,
            pop_mean, pop_var, beta, scale, epsilon)

In [38]:
def build_graph(is_training):
    # Placeholders
    x = tf.placeholder(tf.float32, shape=[None, 784])
    y_ = tf.placeholder(tf.float32, shape=[None, 10])
    # Generate predetermined random weights so the networks are similarly initialized
    w1_initial = tf.Variable(
        tf.truncated_normal([784,100]))
    w2_initial = tf.Variable(
        tf.truncated_normal([100,100]))
    w3_initial = tf.Variable(
        tf.truncated_normal([100,10]))

    b1 = tf.Variable(tf.zeros([100]))
    b2 = tf.Variable(tf.zeros([100]))
    b3 = tf.Variable(tf.zeros([10]))

    # Layer 1
    z1 = tf.matmul(x,w1_initial) + b1
    bn1 = batch_norm_wrapper(z1, is_training)
    l1 = tf.nn.sigmoid(bn1)

    #Layer 2
    z2 = tf.matmul(l1,w2_initial) + b2
    bn2 = batch_norm_wrapper(z2, is_training)
    l2 = tf.nn.relu(bn2)

    # Softmax
    y  = tf.nn.softmax(tf.matmul(l2, w3_initial) + b3)
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

    train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

    correct_prediction = tf.equal(tf.arg_max(y,1),tf.arg_max(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

    return (x, y_), train_step, accuracy, y, tf.train.Saver()

In [ ]:
tf.reset_default_graph()
(x, y_), train_step, accuracy, _, saver = build_graph(is_training=True)

acc = []
num_steps = 10001
batch_size = [64, 128, 256, 512, 1024]
batch_size = 64
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        train_step.run(feed_dict={x: batch_data, y_: batch_labels})
        if step % 50 is 0:
            res = sess.run([accuracy],feed_dict={x: valid_dataset, y_: valid_labels})
            acc.append(res[0])
            print(res[0],end="\t")
    saved_model = saver.save(sess, 'temp-bn-save')

print("Final accuracy:", acc[-1])

0.095	0.0992	0.1024	0.1051	0.1073	0.1112	0.1149	0.1186	0.1217	0.1248	0.129	0.1334	0.1376	0.1421	0.1481	0.1514	0.1554	0.1601	0.1655	0.1707	0.1749	0.1804	0.1852	0.1895	0.1947	0.1992	0.2031	0.2062	0.2104	0.2157	0.219	0.2237	0.228	0.2321	0.2355	0.2401	0.2443	0.248	0.2514	0.2549	0.2584	0.2622	0.2656	0.2693	0.2728	0.2764	0.2797	0.2828	0.2849	0.2878	0.2908	0.2942	0.297	0.3002	0.3031	0.3059	0.308	0.3115	0.3134	0.3166	0.3193	0.3214	0.3233	0.326	0.3294	0.332	0.3345	0.3372	0.3393	0.3415	0.3439	0.346	0.3482	0.3504	0.3529	0.3543	0.3561	0.358	0.3592	0.3611	0.3628	0.3643	0.3661	0.3672	0.3689	0.3713	0.3728	0.3747	0.377	0.3783	0.38	0.3822	0.3835	0.385	0.3865	0.3881	0.3897	0.3911	0.3924	0.3938	0.3953	0.3966	0.3972	0.3987	0.4004	0.402	0.4032	0.405	0.4062	0.4076	0.4089	0.4107	0.4122	0.4137	0.4139	0.4145	0.415	0.4164	0.4179	0.419	0.4197	0.4204	0.4208	0.4217	0.4225	0.4232	0.4241	0.4247	0.4256	0.4267	0.4272	0.4278	0.4283	0.4288	0.4294	0.4304	0.431	0.4322	0.4322	0.4332	0.4352	0.4365	0.4379	0.4391	0.4407	0.441

In [32]:
tf.reset_default_graph()
(x, y_), _, accuracy, y, saver = build_graph(is_training=False)

predictions = []
correct = 0
predictions = []
correct = 0
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    saver.restore(sess, saved_model)
    for i in range(20):
        pred, corr = sess.run([tf.arg_max(y,1), accuracy],
                             feed_dict={x: test_dataset, y_: test_lables})
        correct += corr
        predictions.append(pred[0])
print("PREDICTIONS:", predictions)
print("ACCURACY:", correct/100)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for temp-bn-save
	 [[Node: save/RestoreV2_3 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save/Const_0, save/RestoreV2_3/tensor_names, save/RestoreV2_3/shape_and_slices)]]

Caused by op 'save/RestoreV2_3', defined at:
  File "/Users/sungchulchoi/miniconda3/envs/ml_python/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/sungchulchoi/miniconda3/envs/ml_python/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/sungchulchoi/miniconda3/envs/ml_python/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/sungchulchoi/miniconda3/envs/ml_python/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/sungchulchoi/miniconda3/envs/ml_python/lib/python3